In [1]:
import numpy as np
import pandas as pd
import math
import random as rd
from random import shuffle

In [2]:
# first, define a finite set of alternatives
Alt = {'a','b','c','d','e'}
print(Alt)
#Alt.add('5')
#print(Alt)

{'a', 'b', 'd', 'e', 'c'}


In [3]:
# second, let us define a function capable of generating binary relations
# which does not contain symmetricity 
def binary_relation(A) : # A is to be a set
    AA = A.copy()
    result = set()
    for i in AA : 
        AAA = AA.copy()
        AAA.remove(i)
        for j in AAA:
            if rd.uniform(0,1) > 0.7:
                result.add((i,j))
            else : 
                continue

    result2 = result.copy()
    for m in result :
        for n in result:
            if m[1] == n[0] : 
                result2.add((m[0],n[1]))
            elif m[0] == n[1] :
                result2.add((n[0],m[1]))
    result3 = result2.copy()
    for k in result2 :
        for l in result2:
            if k[0]==l[1] and k[1]==l[0] :
                result3.remove(k)
    return result3

# input : voters' preference relations: as a list
# output : the selected candidate
# assumption : every preference is linear 

In [4]:
def linear_binary_relation(A) : # A: set as well 
    # generates a linear binary relation, as a list
    # this is under developement. so far it acts as a simply random shuffle
    AA = A.copy()
    AAA = list(AA)
    shuffle(AAA)
    return AAA

In [5]:
linear_binary_relation(Alt)

['a', 'c', 'e', 'b', 'd']

In [6]:
def top_scoreing2(D):  # D is a dictionary. keys ; candidates, vals; score
    # returns a pair ; key, val. whose val is the top 
    L = []
    for i in D.keys():
        L.append([i,D[i]])
    rk = L[0][0] ; rv = L[0][1]
    for j in L :
        d = rv - j[1]
        if rv< j[1] :
            rk = j[0] ; rv=j[1]
    return rk,rv

In [7]:
# 1) simple majority voting 
def smv(V, A): # V : voters' preference; 2-dim list, A : candidates' set
    x = {} # dictionary
    for i in A : 
        x[i] = 0
    for i in A:
        for j in V:
            if j[0] is i:  # if the top is i for the agent number j
                x[i] = x[i]+1
#    print(x)
    return x, (top_scoreing2(x))

In [8]:
vv = [linear_binary_relation(Alt) for i in range(0,50)]

#print(vv)

In [9]:
aa,bb = smv(vv,Alt)
print(aa,bb)

{'a': 6, 'b': 6, 'd': 6, 'e': 18, 'c': 14} ('e', 18)


In [10]:
# 2) the Decisive vote 
# given a list of linear preferenes, first vote decides the top and the second.  then the top 2 is to be voted again as a final decision 
# preferenes of the players are supposed not changed in these two votes (is it practical ?)

In [11]:
def top_and_second(V,A) : # V : voters' preference; 2-dim list, A : candidates' set
    a,b = smv(V,A) # a is a dictionary, b ; top
    c = a.pop(b[0])
    sec = top_scoreing2(a)
    return b,sec

In [12]:
def 一位と二位(V,A) : # V : voters' preference; 2-dim list, A : candidates' set
    a,b = smv(V,A) # a is a dictionary, b ; top
    c = a.pop(b[0])
    sec = top_scoreing2(a)
    return b,sec

In [13]:
#vv = [linear_binary_relation(Alt) for i in range(0,400)]
#aaa,bbb = top_and_second(vv,Alt) 
aaa,bbb = 一位と二位(vv,Alt) 

print(aaa,bbb)

('e', 18) ('c', 14)


In [14]:
def find_index(L,a) : # L : list, a, an element of the list
    n=0
    for i in L :
        if a != i : 
            n=n+1
        else :
            break
    return n

In [15]:
def is_prefered(T,L): # T : a 2-tuple, L : a list of candidates
    # returns Boolean value, if T[0]> T[1] then True, else False
    if find_index(L,T[0]) < find_index(L,T[1]):
        return True
    else:
        return False

In [16]:
def binary_vote(T,V): # T ; 2-tuple, V ; list of voters' preferences
    p = len(V)
    nn = 0
    for i in range(0,p):
        if is_prefered(T,V[i]) :
            nn = nn+1
    return ( (T[0], nn), (T[1], p - nn) )

In [17]:
# experiment .....

vv = [linear_binary_relation(Alt) for i in range(0,50)]
tt,uu = top_and_second(vv,Alt)
ss = (tt,uu)
print(ss)
xx = binary_vote((ss[0][0],ss[1][0]),vv)
print(xx)

(('d', 12), ('a', 11))
(('d', 21), ('a', 29))


In [18]:
#vv

In [19]:
# 3) pairwise valuation : 
# first we need to generate a list of pairwise keys to build dictionary

In [20]:
def gen_pairs(A) : # A is a candidates' set
    # generates the list of all pairs on A
    AA = A.copy()
    AAA = list(AA)
    result = []
    for i in range(0,len(AAA)):
        for j in range(i+1,len(AAA)):
            result.append((AAA[i],AAA[j]))
    return result

In [21]:
AltList = list(Alt)

In [22]:
# then, generate a dictionary of pairwise comparison: the numbers of left > right
def pairwise_score(V,A) : # V : voters' preference; 2-dim list, A : candidates' set
    PA = gen_pairs(A) # PA is a list of pairs on A
    result = {}
    for i in PA : 
        sc = 0
        for j in V :
            if is_prefered(i,j):
                sc = sc+1
        result[i]=sc
    return result    

In [23]:
r = pairwise_score(vv,Alt)
print(r)

{('a', 'b'): 25, ('a', 'd'): 29, ('a', 'e'): 33, ('a', 'c'): 27, ('b', 'd'): 26, ('b', 'e'): 28, ('b', 'c'): 23, ('d', 'e'): 28, ('d', 'c'): 25, ('e', 'c'): 20}


In [24]:
# next, generate # of binary win's for each candidate
def pairwise_win(V,A):
    # V : voters' preference
    # A : candidates' set
    ps = pairwise_score(V,A)
    out = {}
    for i in ps.keys():
        if ps[i]<len(V)/2 :
            k = (i[1],i[0])
            s = len(V)-ps[i]
            out[k]=s
        else:
            out[i]=ps[i]
    return out

In [25]:
# Borda rule
def candidates_score(P,A):
    # P : a preference over A(candidates). must be a list of A
    x = P.copy()
    n = len(A)
    out = {}
    for i in range(0,n):
        a = x.pop(0)
        out[a] = n-i
    return out

def borda_scores(V,A) :
    # V : preferences of voters, A : candidates
    out = {}
    counter=0
    for i in range(len(V)):
        out[i] = candidates_score(V[i],A)
    #return out
    out2 = {}
    for a in A :
        out2[a]=0
        for j in out.keys():
            out2[a] = out2[a] + out[j][a]
    return out2

In [26]:
def borda_winner(V,A) :
    # outputs the winner in Borda rule over the preferences V
    X = borda_scores(V,A)
    c = 0
    for i in X.keys():
        if X[i] >= c :
            winner = i
            c = X[i]
        else :
            continue
    return winner,c

In [27]:
def erase_loser(V,A,x): 
    # V : preference relations
    # x : the candidate to erase
    n = len(V[0])
    VV = []
    for i in range(0,len(V)):
        VVV=[]
        for j in range(0,n):
            if V[i][j] == x :
                continue
            VVV.append(V[i][j])
        VV.append(VVV)
    A.remove(x)
    return VV,A

In [28]:
def find_pairwise_loser(V,A): # UNDER CONSTRUCTION
    # findout the pairwise loser among A
    # policy 1 ; the loser against all of the rest should be chosen
    # policy 2 ; if the numbers of losing are the same among candidate i and j, then 
    #   choose candidate who's total losing number is bigger than that of another. 
    pw = pairwise_win(V,A)
    a = {}
    for i in A :
        a[i] = 0
        for j in pw.keys() :
            if j[1] == i :
                a[i] = a[i]+1
            else :
                continue
    aa = a.copy()
    for i in aa.keys():
        for j in aa.keys():
            if i==j:
                continue
            if aa[i]>aa[j]:
                aa[j]=0
    aaa = aa.copy()
    for i in aa.keys():
        if aaa[i] == 0:
            aaa.pop(i)
    return aaa

In [29]:
# experiment 2 .....
Alt = {'a','b','c','d','e','f'}
vv = [linear_binary_relation(Alt) for i in range(0,31)]
tt,uu = top_and_second(vv,Alt)
ss = (tt,uu)
print('top and second:',ss)
xx = binary_vote((ss[0][0],ss[1][0]),vv)
print('pairwise-winning numbers between top-second : ',xx)

print('Borda scoreing winner(w. score):',borda_winner(vv,Alt))

print('pairwise loser(s)(w. # of losing times)',find_pairwise_loser(vv,Alt))

top and second: (('d', 9), ('e', 8))
pairwise-winning numbers between top-second :  (('d', 14), ('e', 17))
Borda scoreing winner(w. score): ('e', 122)
pairwise loser(s)(w. # of losing times) {'a': 5}


In [74]:
# repeated experiment : 2
Alt = {'a','b','c','d','e','f'}
result = {}
result['top and second:']=[]
result['pairwise-winning numbers between top-second :'] = []
result['Borda scoreing winner(w. score):'] = []
result['pairwise loser(s)'] = []

# num. of experiment times
nexp = 200
# voters' number
vn = 31

for i in range(0,nexp):
    vv = [linear_binary_relation(Alt) for i in range(0,vn)]
    tt,uu = top_and_second(vv,Alt)
    ss = (tt,uu)
    xx = binary_vote((ss[0][0],ss[1][0]),vv)
    result['top and second:'].append(ss)
    result['pairwise-winning numbers between top-second :'].append(xx)
    result['Borda scoreing winner(w. score):'].append(borda_winner(vv,Alt))
    result['pairwise loser(s)'].append(find_pairwise_loser(vv,Alt))
pd.set_option('display.max_rows', 30)
pd.DataFrame(result)

,top and second:,pairwise-winning numbers between top-second :,Borda scoreing winner(w. score):,pairwise loser(s)
0,"((a, 10), (e, 5))","((a, 18), (e, 13))","(a, 115)",{'b': 5}
1,"((e, 8), (d, 6))","((e, 15), (d, 16))","(c, 119)",{'b': 4}
2,"((d, 7), (b, 6))","((d, 16), (b, 15))","(b, 115)","{'a': 4, 'e': 4}"
3,"((d, 9), (c, 6))","((d, 22), (c, 9))","(d, 138)","{'e': 4, 'c': 4}"
4,"((a, 7), (e, 7))","((a, 16), (e, 15))","(c, 123)",{'d': 5}
...,...,...,...,...
195,"((a, 11), (d, 6))","((a, 16), (d, 15))","(a, 123)",{'e': 5}
196,"((d, 8), (a, 5))","((d, 19), (a, 12))","(d, 121)",{'b': 5}
197,"((c, 10), (d, 6))","((c, 19), (d, 12))","(c, 127)",{'f': 5}
198,"((a, 7), (d, 7))","((a, 13), (d, 18))","(c, 112)",{'f': 4}


In [75]:
df_result.to_csv('kyouyou_jikken.csv', encoding='utf-8')

In [34]:
import csv

In [78]:
# repeated experiment : 2-2
Alt = {'a','b','c','d','e','f'}
result = []

fields = ['no:','top and second:','pairwise-winning numbers between top-second :','Borda scoreing winner(w. score):','pairwise loser(s)']

# num. of experiment times
nexp = 300
# voters' number
vn = 31

for i in range(0,nexp):
    r = {}
    vv = [linear_binary_relation(Alt) for i in range(0,vn)]
    r['no:']=i+1
    tt,uu = top_and_second(vv,Alt)
    ss = (tt,uu)
    r['top and second:'] = ss
    xx = binary_vote((ss[0][0],ss[1][0]),vv)
    r['pairwise-winning numbers between top-second :'] = xx
    r['Borda scoreing winner(w. score):'] = borda_winner(vv,Alt)
    r['pairwise loser(s)'] = find_pairwise_loser(vv,Alt)
    result.append(r)

In [79]:
#result
csv_file = "experiment2.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        writer.writeheader()
        for data in result:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [269]:
#find_pairwise_loser(vv,Alt)

In [215]:
pairwise_win(vv,Alt)

{('c', 'a'): 16,
 ('a', 'f'): 17,
 ('b', 'a'): 16,
 ('a', 'e'): 19,
 ('a', 'd'): 21,
 ('c', 'f'): 19,
 ('c', 'b'): 17,
 ('c', 'e'): 17,
 ('c', 'd'): 18,
 ('b', 'f'): 18,
 ('f', 'e'): 17,
 ('f', 'd'): 16,
 ('b', 'e'): 19,
 ('b', 'd'): 18,
 ('e', 'd'): 20}

In [112]:
vv,Alt = erase_loser(vv,Alt,'e')

In [115]:
# experiment 3
tt,uu = top_and_second(vv,Alt)
ss = (tt,uu)
print(ss)
xx = binary_vote((ss[0][0],ss[1][0]),vv)
print(xx)

borda_winner(vv,Alt)

(('d', 15), ('a', 14))
(('d', 26), ('a', 24))


('c', 129)

In [116]:
pairwise_win(vv,Alt)

{('a', 'c'): 26,
 ('d', 'a'): 26,
 ('b', 'a'): 26,
 ('c', 'd'): 28,
 ('c', 'b'): 27,
 ('d', 'b'): 25}

In [117]:
vv,Alt = erase_loser(vv,Alt,'a')

In [118]:
# experiment 4
tt,uu = top_and_second(vv,Alt)
ss = (tt,uu)
print(ss)
xx = binary_vote((ss[0][0],ss[1][0]),vv)
print(xx)

borda_winner(vv,Alt)

(('c', 18), ('d', 17))
(('c', 28), ('d', 22))


('c', 105)

In [119]:
pairwise_win(vv,Alt)

{('c', 'd'): 28, ('d', 'b'): 25, ('c', 'b'): 27}

In [120]:
vv,Alt = erase_loser(vv,Alt,'b')

In [121]:
# experiment 5
tt,uu = top_and_second(vv,Alt)
ss = (tt,uu)
print(ss)
xx = binary_vote((ss[0][0],ss[1][0]),vv)
print(xx)

borda_winner(vv,Alt)

(('c', 28), ('d', 22))
(('c', 28), ('d', 22))


('c', 78)

In [122]:
pairwise_win(vv,Alt)

{('c', 'd'): 28}

In [ ]:
# coalition : how to implement ??
#xxx = np.array([])
#type(xxx)
#Alt = {'a','b','c','d','e'}
# 1) top nth is to be fixed
C1 = [['a','b','c','',''],
      ['d','e','','',''],
      ['c','d','','','']
     ]
# 2) bottom nth is to be fixed 
C2 = [['','','','b','a' ],
      ['','','d','e','c'],
      ['','','','c','d']
     ]
# 3) part of the order is fixed 
C3 = [['e','d','a'],
      ['c','b','e']
     ]

In [ ]:
aaaa=['a','b','c']
aaaa.remove('c')

In [ ]:
def diff(f, s): # implementation of "diff"
    ss = set(s)
    return [i for i in f if i not in ss]

In [ ]:
#rrr=diff(Alt,C1[0])

In [ ]:
A=Alt.copy()
diff(list(A),aaaa)

In [30]:
def generate_top_n(C,A): # C is the coalition definition, A is the candidates' set
    # T is the coalition lists 
    j = 0
    AA = A.copy()
    AAA=list(AA)
    shuffle(AAA)
    r = []
    j=0
    for i in AA:
        if i == C[j]:
            r.append(i)
            AAA.remove(i)
            j=j+1
    return r
            

In [31]:
generate_top_n(C1[0],Alt)